<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/17_Aug_2024_Final1_Prediction_steps_LF_12_30pm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import requests
import os
from datetime import datetime, timedelta, time as dt_time
import pytz
import time
from difflib import SequenceMatcher
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime, time as dt_time
import time

from google.colab import drive
drive.mount('/content/drive')

# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())
from google.colab import drive
drive.mount('/content/drive')

# Read the Excel file
excel_file_path = '/content/drive/MyDrive/Files/QSI points.xlsx'
df = pd.read_excel(excel_file_path)
df_sheet2 = pd.read_excel(excel_file_path, sheet_name='Sheet2')
output_dir = '/content/drive/My Drive/Files/'

# Load the training data from Google Drive
file_path_train = '/content/drive/My Drive/Files/Training_set.csv'
df_train = pd.read_csv(file_path_train)

# Get the lineID from the user
lineID = input("Please enter the lineID: ")

# Check if the necessary column is present
if 'Route_Dir_QSI_No' not in df.columns:
    print("The 'Route_Dir_QSI_No' column is not present in the provided file.")
else:
    # Convert the lineID to uppercase to ensure case-insensitivity
    lineID = lineID.upper()

    # Convert the 'Route_Dir_QSI_No' column to uppercase for comparison
    df['Route_Dir_QSI_No'] = df['Route_Dir_QSI_No'].str.upper()

    # Normalize the stop names in the DataFrame
    df['STOP_NAME'] = df['STOP_NAME'].apply(normalize_stop_name)

    # Create regular expressions for filtering
    pattern_A = f"^{lineID}_A\\d+$"  # Regular expression for lineID_A**
    pattern_B = f"^{lineID}_B\\d+$"  # Regular expression for lineID_B**

    # Filter rows where the 'Route_Dir_QSI_No' column matches the pattern
    filtered_df_A = df[df['Route_Dir_QSI_No'].str.match(pattern_A, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]
    filtered_df_B = df[df['Route_Dir_QSI_No'].str.match(pattern_B, na=False)][['Route_Dir_QSI_No', 'STOP_NAME']]

    # Function to fetch and process route data from TfL API
    def fetch_and_process_route_data(route_type, pattern, filtered_df):
        api_url = f"https://api.tfl.gov.uk/Line/{lineID}/Route/Sequence/{route_type}"
        response = requests.get(api_url)

        results_list = []

        if response.status_code == 200:
            route_data = response.json()

            # Iterate through each stop in the route data
            for stop in route_data['stopPointSequences'][0]['stopPoint']:
                stop_name_api = normalize_stop_name(stop['name'])
                stop_id = stop['id']

                # Check if the stop_name_api exists in the filtered DataFrame for the correct direction
                matched_row = filtered_df[(filtered_df['STOP_NAME'] == stop_name_api) &
                                          (filtered_df['Route_Dir_QSI_No'].str.match(pattern))]

                if not matched_row.empty:
                    route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                    results_list.append({
                        'Route_Dir_QSI_No': route_dir_qsi_no,
                        'STOP_Name': stop['name'],
                        'ID': stop_id
                    })
                else:
                    # If exact match not found, try partial matching based on words before and after '/'
                    api_stop_name_parts = stop_name_api.split('/')
                    for index, row in filtered_df.iterrows():
                        df_stop_name_parts = row['STOP_NAME'].split('/')
                        for api_part in api_stop_name_parts:
                            for df_part in df_stop_name_parts:
                                if SequenceMatcher(None, df_part.strip(), api_part.strip()).ratio() > 0.8:
                                    matched_row = pd.DataFrame([row])
                                    break
                            if not matched_row.empty:
                                break
                        if not matched_row.empty:
                            break

                    if not matched_row.empty:
                        route_dir_qsi_no = matched_row.iloc[0]['Route_Dir_QSI_No']
                        results_list.append({
                            'Route_Dir_QSI_No': route_dir_qsi_no,
                            'STOP_Name': stop['name'],
                            'ID': stop_id
                        })
        else:
            print(f"Failed to fetch route sequence data from TfL API for {route_type} route. Status code: {response.status_code}")

        return results_list

    # Fetch and process outbound route data for _A**
    matched_results_A = fetch_and_process_route_data('outbound', pattern_A, filtered_df_A)

    # Fetch and process inbound route data for _B**
    matched_results_B = fetch_and_process_route_data('inbound', pattern_B, filtered_df_B)

    # Create DataFrames from the matched results for each direction
    matched_results_df_A = pd.DataFrame(matched_results_A)
    matched_results_df_B = pd.DataFrame(matched_results_B)

    # Function to remove partial matches if exact matches are found
    def remove_partial_matches(exact_df, matched_df):
        for index, row in exact_df.iterrows():
            exact_stop_name = row['STOP_NAME']
            route_dir_qsi_no = row['Route_Dir_QSI_No']
            # Find exact matches in matched_df
            exact_matches = matched_df[(matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                       (matched_df['STOP_Name'].apply(normalize_stop_name) == exact_stop_name)]
            if not exact_matches.empty:
                # Remove partial matches
                matched_df = matched_df[~((matched_df['Route_Dir_QSI_No'] == route_dir_qsi_no) &
                                          (matched_df['STOP_Name'].apply(normalize_stop_name) != exact_stop_name))]
        return matched_df

    # Remove partial matches for direction A
    matched_results_df_A = remove_partial_matches(filtered_df_A, matched_results_df_A)

    # Remove partial matches for direction B
    matched_results_df_B = remove_partial_matches(filtered_df_B, matched_results_df_B)

    # Remove duplicate stop names with the same Route_Dir_QSI_No and different IDs
    matched_results_df_A = matched_results_df_A.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')
    matched_results_df_B = matched_results_df_B.drop_duplicates(subset=['Route_Dir_QSI_No', 'STOP_Name'], keep='first')

    # Read the bus-stops.csv file
    #bus_stops_file_path = '/content/drive/MyDrive/Report/bus-stops.csv'
    bus_stops_df = pd.read_excel(excel_file_path, sheet_name='Sheet3')

    # Normalize the stop names in the bus stops DataFrame
    bus_stops_df['Stop_Name'] = bus_stops_df['Stop_Name'].apply(normalize_stop_name)

    # Match QSI point IDs with bus stop codes
    def match_qsi_with_bus_stops(matched_df, bus_stops_df):
        matched_results = []
        for index, row in matched_df.iterrows():
            stop_id = row['ID']
            matched_stop = bus_stops_df[bus_stops_df['Naptan_Atco'] == stop_id]
            if not matched_stop.empty:
                bus_stop_code = matched_stop.iloc[0]['Bus_Stop_Code']
                matched_results.append({
                    'Route_Dir_QSI_No': row['Route_Dir_QSI_No'],
                    'STOP_Name': row['STOP_Name'],
                    'ID': row['ID'],
                    'Bus_Stop_Code': bus_stop_code
                })
        return pd.DataFrame(matched_results)

    # Perform the matching for direction A
    matched_results_df_A = match_qsi_with_bus_stops(matched_results_df_A, bus_stops_df)

    # Perform the matching for direction B
    matched_results_df_B = match_qsi_with_bus_stops(matched_results_df_B, bus_stops_df)

    # Print the matched results for direction A
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_A\033[0m\n")
    print(matched_results_df_A[['Route_Dir_QSI_No', 'STOP_Name', 'ID', 'Bus_Stop_Code']])

    # Print the matched results for direction B
    print(f"\n\n\033[1m\033[4mQSI stop points for direction {lineID}_B\033[0m\n")
    print(matched_results_df_B[['Route_Dir_QSI_No', 'STOP_Name', 'ID', 'Bus_Stop_Code']])

    # Concatenate the matched results DataFrames for directions A and B
    combined_df = pd.concat([matched_results_df_A, matched_results_df_B], ignore_index=True)

    # Display the combined DataFrame
    print("\n\n\033[1m\033[4mCombined QSI stop points for directions A and B\033[0m\n")
    print(combined_df[['Route_Dir_QSI_No', 'STOP_Name', 'ID', 'Bus_Stop_Code']])


Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Please enter the lineID: d7


QSI stop points for direction D7_A

  Route_Dir_QSI_No                    STOP_Name          ID  Bus_Stop_Code
0            D7_A1   Poplar / All Saints Church  490011107G          73923
1            D7_A2               Stewart Street  490013513S          47950
2            D7_A3       Island Gardens Station  490002048Z          76947
3            D7_A4  Arnhem Wharf Primary School  490006092N          50948
4            D7_A5         East India Dock Road  490004584N          47475


QSI stop points for direction D7_B

  Route_Dir_QSI_No                    STOP_Name          ID  Bus_Stop_Code
0            D7_B1  Mile End Station / Bow Road  490015151H          48439
1            D7_B2         East India Dock Road  490004584S          56224
2            D7_B3         Canary Wharf Station  490000038F       

In [3]:
# Function to normalize stop names
def normalize_stop_name(name):
    return ' '.join(name.lower().split())

# Function to fetch data from the TfL API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to fetch the current day of the week
def get_day_of_week():
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    return now.strftime('%A')  # %A gives full weekday name (e.g., 'Monday')

# Function to select the preferred schedule name based on the current day of the week
def select_preferred_schedule(schedule_names_dict, day_of_week):
    if day_of_week.lower() in ['monday', 'tuesday', 'wednesday', 'thursday']:
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Thursday', 'Monday to Friday']
    elif day_of_week.lower() == 'friday':
        preferred_schedule_names = ['Mon-Fri Schooldays', 'Monday to Friday', 'Friday']
    elif day_of_week.lower() == 'saturday':
        preferred_schedule_names = ['Saturday']
    elif day_of_week.lower() == 'sunday':
        preferred_schedule_names = ['Sunday']
    else:
        preferred_schedule_names = [day_of_week]

    for preferred_name in preferred_schedule_names:
        if preferred_name in schedule_names_dict:
            return preferred_name
    return None


# Function to display the timetable for each stop point
def display_timetable_for_stop_points(combined_df, lineID):
    timetable_dict = {}

    # Fetch the current time in BST
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    current_hour = now.hour
    current_minute = now.minute

    # Get the current day of the week
    day_of_week = get_day_of_week()

    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        route_dir_qsi_no = row['Route_Dir_QSI_No']
        stop_name = row['STOP_Name']

        # Determine the direction
        if f"{lineID}_A" in route_dir_qsi_no:
            direction = 'outbound'
        elif f"{lineID}_B" in route_dir_qsi_no:
            direction = 'inbound'
        else:
            continue  # Skip if direction cannot be determined

        # Fetch timetable data for the stop point
        url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
        data = fetch_data(url)

        # Extract schedule names
        schedule_names_dict = extract_schedule_names(data)

        # Select the preferred schedule name based on the current day of the week

        selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

        if selected_schedule_name:
            # Categorize the timetable into hourly slots
            timetable = schedule_names_dict[selected_schedule_name]
            slots = categorize_into_slots(timetable)

            # Prepare a DataFrame for the timetable for the specified hours
            timetable_list = []

            for hour in range(24):
                for journey in slots[hour]:
                    journey_hour = str(journey['hour']).zfill(2)
                    journey_minute = str(journey['minute']).zfill(2)
                    time = f"{journey_hour}:{journey_minute}"
                    timetable_list.append({
                        'Line': lineID,
                        #'Stop Point': stop_name,
                        'Stop Point ID': stop_point_id,
                        'Direction': direction,
                        'Scheduled Time': time
                    })

            timetable_df = pd.DataFrame(timetable_list)
            timetable_dict[stop_point_id] = timetable_df

    return timetable_dict


# Display the timetable
timetable_dict = display_timetable_for_stop_points(combined_df, lineID)


# Save the timetable dictionary to files
for stop_point_id, timetable_df in timetable_dict.items():
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    timetable_df.to_csv(f'{output_dir}timetable_{stop_point_id}.csv', index=False)
    # Save the DataFrame to a CSV file
    print(f"\n\n\033[1m\033[4mTimetable for {stop_point_id}:\033[0m\n")
    print(timetable_df.to_string(index=False))




Timetable for 490011107G:

Line Stop Point ID Direction Scheduled Time
  D7    490011107G  outbound          04:32
  D7    490011107G  outbound          04:52
  D7    490011107G  outbound          05:12
  D7    490011107G  outbound          05:32
  D7    490011107G  outbound          05:52
  D7    490011107G  outbound          06:12
  D7    490011107G  outbound          06:32
  D7    490011107G  outbound          06:47
  D7    490011107G  outbound          07:02
  D7    490011107G  outbound          07:17
  D7    490011107G  outbound          07:32
  D7    490011107G  outbound          07:47
  D7    490011107G  outbound          07:59
  D7    490011107G  outbound          08:11
  D7    490011107G  outbound          08:23
  D7    490011107G  outbound          08:34
  D7    490011107G  outbound          08:46
  D7    490011107G  outbound          08:58
  D7    490011107G  outbound          09:09
  D7    490011107G  outbound          09:21
  D7    490011107G  outbound          09:32
  D

In [4]:
def get_timetable_for_stop_point(lineID, stop_point_id, combined_df):
    direction = 'outbound' if combined_df[combined_df['ID'] == stop_point_id]['Route_Dir_QSI_No'].iloc[0].startswith(lineID + '_A') else 'inbound'
    url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
    data = fetch_data(url)

    # Extract schedule names
    schedule_names_dict = extract_schedule_names(data)
    day_of_week = get_day_of_week()

    # Select the preferred schedule name
    selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

    if selected_schedule_name:
        timetable = schedule_names_dict[selected_schedule_name]
        slots = categorize_into_slots(timetable)

        # Prepare a DataFrame for the timetable for the specified hours
        timetable_list = []
        for hour in range(24):
            for journey in slots[hour]:
                journey_hour = str(journey['hour']).zfill(2)
                journey_minute = str(journey['minute']).zfill(2)
                time = f"{journey_hour}:{journey_minute}"
                timetable_list.append({
                    'Line': lineID,
                    'Stop Point ID': stop_point_id,
                    'Direction': direction,
                    'Scheduled Time': time
                })

        return pd.DataFrame(timetable_list)
    else:
        return pd.DataFrame()  # Return empty DataFrame if no schedule is found


In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta, time as dt_time

# Function to fetch arrival predictions
def fetch_arrival_predictions(line_id, stop_point_id, direction):
    try:
        base_url = f"https://api.tfl.gov.uk/Line/{line_id}/Arrivals/{stop_point_id}"
        params = {'direction': direction}
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        if len(data) == 0:
            return pd.DataFrame(), None  # No data available
        station_name = data[0]['stationName']
        predictions = []
        for item in data:
            arrival_time = datetime.strptime(item['expectedArrival'], '%Y-%m-%dT%H:%M:%SZ')
            arrival_time_bst = arrival_time + timedelta(hours=1)
            predictions.append({
                'Line': item['lineName'],
                'Vehicle ID': item['vehicleId'],
                'Stop Point': stop_point_id,
                'Direction': direction,
                'Expected Arrival (BST)': arrival_time_bst,
                'Expected Arrival (HM)': arrival_time_bst.strftime('%H:%M')
            })
        df = pd.DataFrame(predictions)
        df = df.sort_values(by='Expected Arrival (BST)', ascending=True)
        df['Expected Arrival (BST)'] = pd.to_datetime(df['Expected Arrival (BST)'])  # Convert to datetime
        df['Expected Arrival (HM)'] = pd.to_datetime(df['Expected Arrival (HM)'], format='%H:%M')
        df['Gap'] = df['Expected Arrival (HM)'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds() / 60
        df['2_Gap'] = (df['Gap'] * 2).round(2)
        df['Gap_Sq'] = (df['Gap'] * df['Gap']).round(2)
        return df, station_name
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None, None

def fetch_current_hour_swt(route_swt_df):
    current_hour = datetime.now(pytz.timezone('Europe/London')).hour

    current_swt = route_swt_df[route_swt_df['Hour'] == current_hour]
    if not current_swt.empty:
        route_swt_a = current_swt['Route SWT A'].values[0]
        route_swt_b = current_swt['Route SWT B'].values[0]

        # Calculate double headway
        double_headway_a = route_swt_a * 2
        double_headway_b = route_swt_b * 2 * 2

        print(f"Route SWT A for the current hour ({current_hour}): {route_swt_a}")
        print(f"Route SWT B for the current hour ({current_hour}): {route_swt_b}")
        print(f"Double Headway for Route A: {double_headway_a}")
        print(f"Double Headway for Route B: {double_headway_b}")
        return double_headway_a, double_headway_b
    else:
        print(f"No SWT data available for the current hour ({current_hour}).")

def check_gaps_against_double_headway(cumulative_dataframes, double_headway_a, double_headway_b, combined_df, lineID):
    results = []

    # Create a mapping from ID to Bus_Stop_Code
    id_to_code = combined_df.set_index('ID')['Bus_Stop_Code'].to_dict()

    # Loop through each stop point ID in the cumulative dataframes
    for stop_point_id, cumulative_df in cumulative_dataframes.items():
        direction = 'A' if combined_df[combined_df['ID'] == stop_point_id]['Route_Dir_QSI_No'].iloc[0].startswith(lineID + '_A') else 'B'
        double_headway = double_headway_a if direction == 'A' else double_headway_b

        # Check if any 'Gap' is greater than the corresponding double headway
        gaps_exceeding_headway = cumulative_df[cumulative_df['Gap'] > double_headway]

        if not gaps_exceeding_headway.empty:
            # Extract the relevant Vehicle ID and Stop Point
            for _, row in gaps_exceeding_headway.iterrows():
                results.append({ 'Line': lineID,
                                'Vehicle ID': row['Vehicle ID'],
                                'Stop Point': row['Stop Point']
                              })

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    if not results_df.empty:
        # Add Bus_Stop_Code to results_df
        results_df['Bus_Stop_Code'] = results_df['Stop Point'].map(id_to_code)
    if results_df.empty:
        print("\nNo Vehicle IDs and Stop Points where Gap exceeds Double Headway.")
    else:
        print("\nVehicle IDs and Stop Points where Gap exceeds Double Headway:")
        print(results_df.to_string(index=False))

    return results_df

def main():

    # Function to calculate Scheduled Wait Time (SWT) for all hours
    def calculate_swt_for_all_hours(slots):
        swt_per_hour = []
        for hour in range(24):
            total_buses = len(slots[hour])
            scheduled_wait_time = 60 / (total_buses * 2) if total_buses > 0 else None  # Use None to indicate no buses
            swt_per_hour.append((scheduled_wait_time, total_buses))
        return swt_per_hour

    # Main logic to fetch and calculate SWT
    def main(combined_df, lineID):
        bst = pytz.timezone('Europe/London')
        swt_data = {
            'Route_Dir_QSI_No': [],
            'ID': [],
        }
        # Initialize keys for all 24 hours in the dictionary
        for hour in range(24):
            swt_data[f'SWT_{hour}'] = []
            swt_data[f'Sch_{hour}'] = []

        # Update current time and hour
        current_time = datetime.now(bst)
        day_of_week = get_day_of_week()

        # Store selected schedule name to ensure it's printed only once
        selected_schedule_name = None
        printed_schedule_name = False

        for index, row in combined_df.iterrows():
            stop_point_id = row['ID']
            route_dir_qsi_no = row['Route_Dir_QSI_No']

            if f"{lineID}_A" in route_dir_qsi_no:
                direction = 'outbound'
            elif f"{lineID}_B" in route_dir_qsi_no:
                direction = 'inbound'
            else:
                continue

            url = f'https://api.tfl.gov.uk/Line/{lineID}/Timetable/{stop_point_id}?direction={direction}'
            data = fetch_data(url)

            schedule_names_dict = extract_schedule_names(data)

            if not selected_schedule_name:
                selected_schedule_name = select_preferred_schedule(schedule_names_dict, day_of_week)

            if selected_schedule_name and not printed_schedule_name:
                print(f"\n\033[1m\033[4mToday is {day_of_week}. The selected Schedule name is {selected_schedule_name}.\033[0m")
                printed_schedule_name = True

            if selected_schedule_name:
                timetable = schedule_names_dict[selected_schedule_name]
                slots = categorize_into_slots(timetable)

                # Calculate SWT for all hours
                swt_per_hour = calculate_swt_for_all_hours(slots)

                # Store SWT data for all hours
                swt_data['Route_Dir_QSI_No'].append(route_dir_qsi_no)
                swt_data['ID'].append(stop_point_id)
                for hour in range(24):
                    swt, total_buses = swt_per_hour[hour]
                    swt_data[f'SWT_{hour}'].append(swt)
                    swt_data[f'Sch_{hour}'].append(total_buses)

        # Create DataFrame for SWT data
        swt_df = pd.DataFrame(swt_data)

        # Calculate Route SWT for each hour for directions A and B
        route_swt_data = { 'Line' : [] ,
            'Hour': [],
            'Route SWT A': [],
            'Route SWT B': []
        }

        for hour in range(24):
            # Calculate Route SWT for direction A
            swt_a = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_A')]
            valid_swt_a = swt_a[pd.notna(swt_a[f'SWT_{hour}'])]  # Filter out NaN values
            weighted_sum_a = sum(valid_swt_a[f'SWT_{hour}'] * valid_swt_a[f'Sch_{hour}']) if not valid_swt_a.empty else 0
            total_buses_a = sum(valid_swt_a[f'Sch_{hour}']) if not valid_swt_a.empty else 0
            route_swt_a = round(weighted_sum_a / total_buses_a, 2) if total_buses_a > 0 else None


            # Calculate Route SWT for direction B
            swt_b = swt_df[swt_df['Route_Dir_QSI_No'].str.contains(f'{lineID}_B')]
            valid_swt_b = swt_b[pd.notna(swt_b[f'SWT_{hour}'])]  # Filter out NaN values
            weighted_sum_b = sum(valid_swt_b[f'SWT_{hour}'] * valid_swt_b[f'Sch_{hour}']) if not valid_swt_b.empty else 0
            total_buses_b = sum(valid_swt_b[f'Sch_{hour}']) if not valid_swt_b.empty else 0
            route_swt_b = round(weighted_sum_b / total_buses_b, 2) if total_buses_b > 0 else None

            route_swt_data['Line'].append(lineID)
            route_swt_data['Hour'].append(hour)
            route_swt_data['Route SWT A'].append(route_swt_a)
            route_swt_data['Route SWT B'].append(route_swt_b)

        route_swt_df = pd.DataFrame(route_swt_data)
        return route_swt_df

    route_swt_df = main(combined_df, lineID)
    # Example dictionary to hold cumulative dataframes for each stop point

    cumulative_dataframes = {}
    cumulative_dataframes_new = {}
    # Dictionary to hold the number of buses observed per stop point
    buses_observed = {}
    buses_observed_new = {}
    buses_observed_new2 = {}
    # DataFrame to store Route AWT data
    route_awt_df = pd.DataFrame(columns=['Hour', 'Route AWT A', 'Route AWT B'])
    route_awt_df_new = pd.DataFrame(columns=['Hour', 'Route AWT A', 'Route AWT B'])
    route_awt_df_new2 = pd.DataFrame(columns=['Hour', 'Route AWT A', 'Route AWT B'])
    # Loop through unique stop points in combined_df
    for index, row in combined_df.iterrows():
        stop_point_id = row['ID']
        direction = 'outbound' if row['Route_Dir_QSI_No'].startswith(lineID + '_A') else 'inbound'

        cumulative_df = pd.DataFrame(columns=[
            'Line', 'Vehicle ID', 'Stop Point', 'Direction',
            'Expected Arrival (BST)', 'Expected Arrival (HM)',
            'Gap', '2_Gap', 'Gap_Sq'
        ])
        cumulative_df_new = cumulative_df
        cumulative_dataframes[stop_point_id] = cumulative_df  # Initialize cumulative dataframe
        cumulative_dataframes_new[stop_point_id] = cumulative_df_new
        buses_observed[stop_point_id] = (0, 0, 0, 0)  # Initialize with zero values
        buses_observed_new[stop_point_id] = (0, 0, 0, 0)  # Initialize with zero values
        buses_observed_new2[stop_point_id] = (0, 0, 0, 0)  # Initialize with zero values
    while True:
        for stop_point_id, cumulative_df in cumulative_dataframes.items():
            direction = 'outbound' if combined_df[combined_df['ID'] == stop_point_id]['Route_Dir_QSI_No'].iloc[0].startswith(lineID + '_A') else 'inbound'

            arrival_predictions_df, station_name = fetch_arrival_predictions(lineID, stop_point_id, direction)


            if arrival_predictions_df is not None and not arrival_predictions_df.empty:
                current_hour = datetime.now(pytz.timezone('Europe/London')).hour

                for _, row in arrival_predictions_df.iterrows():
                    vehicle_id = row['Vehicle ID']

                    mask = cumulative_df['Vehicle ID'] == vehicle_id

                    if cumulative_df[mask].empty:
                        # If vehicle ID is not present in cumulative DataFrame, append the row
                        cumulative_df = pd.concat([cumulative_df, row.to_frame().T], ignore_index=True)
                    else:
                        # If vehicle ID is present, overwrite the row
                        cumulative_df.loc[mask, ['Line', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Expected Arrival (HM)', 'Gap', '2_Gap', 'Gap_Sq']] = row[['Line', 'Stop Point', 'Direction', 'Expected Arrival (BST)', 'Expected Arrival (HM)', 'Gap', '2_Gap', 'Gap_Sq']].values

                # Convert to datetime
                cumulative_df['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df['Expected Arrival (BST)'])
                cumulative_df['Expected Arrival (HM)'] = pd.to_datetime(cumulative_df['Expected Arrival (HM)'], format='%H:%M')

                # Sort the DataFrame
                cumulative_df = cumulative_df.sort_values(by='Expected Arrival (BST)', ascending=True).reset_index(drop=True)

                # Calculate gaps
                cumulative_df['Gap'] = (cumulative_df['Expected Arrival (BST)'].diff().dt.total_seconds() / 60).round(2)
                cumulative_df.loc[0, 'Gap'] = 0  # First row gap should be zero
                cumulative_df['2_Gap'] = (cumulative_df['Gap'] * 2).round(2)
                cumulative_df['Gap_Sq'] = (cumulative_df['Gap'] * cumulative_df['Gap']).round(2)
                #print(cumulative_df)
                # Update number of buses observed in the current hour
                num_buses_observed = len(cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour]['Vehicle ID'].unique())
                total_Gap_Sq = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour]['Gap_Sq'].sum()
                total_2_Gap = cumulative_df[cumulative_df['Expected Arrival (BST)'].dt.hour == current_hour]['2_Gap'].sum()
                AWT = round(total_Gap_Sq / total_2_Gap, 2) if total_2_Gap > 0 else 0
                buses_observed[stop_point_id] = (num_buses_observed, total_Gap_Sq, total_2_Gap, AWT)

                print(f"\nArrival Predictions for stop point {stop_point_id} ({station_name}):")
                print(arrival_predictions_df.to_string(index=False))
                print("\nCumulative DataFrame 1:")
                print(cumulative_df.to_string(index=False))
                print(f"\nNumber of buses observed in the current hour: {num_buses_observed}")

                timetable_df = get_timetable_for_stop_point(lineID, stop_point_id, combined_df)
                #if not timetable_df.empty:
                  #print(f"\nTimetable for stop point {stop_point_id}:")
                  #print(timetable_df.to_string(index=False))

            else:
                print("No arrival predictions available.")

            print("Refreshing data in 30 seconds...\n")
            time.sleep(30)

            # Update cumulative dataframe in dictionary
            cumulative_dataframes[stop_point_id] = cumulative_df
            cumulative_df_new = cumulative_df
            cumulative_dataframes_new[stop_point_id] = cumulative_df_new
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            cumulative_df.to_csv(f'{output_dir}cumulative_data_stop_{stop_point_id}.csv', index=False)

            if cumulative_df_new.empty:
                print("No data available in cumulative_df_new.")
                first_row = None  # Assign None or handle the absence of data appropriately
            else:
                first_row = cumulative_df_new.iloc[0]
                #print(first_row)
                # Retrieve the 'Expected Arrival (BST)' value and extract the time part
                expected_arrival_time = first_row['Expected Arrival (BST)'].time()
                #print(f"Expected arrival time: {expected_arrival_time}")

                #timetable_dict[stop_point_id] = timetable_df
                #print(f"timetable of {stop_point_id}: {timetable_df}")

                # Create a function to convert 'Scheduled Time' to a datetime.time object
                def to_time(time_str):
                    return datetime.strptime(time_str, '%H:%M').time()

                # Apply the function to the 'Scheduled Time' column
                timetable_df['Scheduled Time Only'] = timetable_df['Scheduled Time'].apply(to_time)

                # Filter the rows where 'Scheduled Time Only' is greater than 'expected_arrival_time'
                filtered_df = timetable_df[timetable_df['Scheduled Time Only'] > expected_arrival_time]

                # Select the top 10 'Scheduled Time Only' values
                top_20_scheduled_time = filtered_df['Scheduled Time Only'].head(20)
                #print(f"top 20 schedule: {top_20_scheduled_time}")

                # Find the first row's scheduled time from top_20_scheduled_time
                first_time = top_20_scheduled_time.iloc[0]

                # Find the index of this time in the original timetable_df
                index_of_first_time = timetable_df[timetable_df['Scheduled Time Only'] == first_time].index[0]

                # Get the scheduled time immediately above it
                if index_of_first_time > 0:
                    previous_time = timetable_df.iloc[index_of_first_time - 1]['Scheduled Time Only']
                else:
                    # If it's the first element, there's no previous time
                    previous_time = None

                # Create a new DataFrame to include the previous time
                if previous_time:
                    top_21_scheduled_time = pd.concat([
                        pd.Series([previous_time], name='Scheduled Time Only'),
                        top_20_scheduled_time
                    ]).drop_duplicates().reset_index(drop=True)
                else:
                    top_21_scheduled_time = top_20_scheduled_time.reset_index(drop=True)

                # Print the result
                #print("Updated top 20 'Scheduled Time Only' values including the time immediately above the first element:")
                #print(top_21_scheduled_time)

                # Number of rows in cumulative_df
                num_rows_cumulative_new = len(cumulative_df_new)

                # Fetch the first 'num_rows_cumulative' scheduled times from top_21_scheduled_time
                scheduled_times_to_add = top_21_scheduled_time.head(num_rows_cumulative_new)

                # Add these scheduled times as a new column in cumulative_df
                cumulative_df_new['Scheduled Time'] = scheduled_times_to_add.values

                # Print the updated cumulative_df
                #print("\nUpdated cumulative_df with new 'Scheduled Time' column:")
                #print(cumulative_df_new)

                # Define the BST timezone
                bst = pytz.timezone('Europe/London')

                # Get the current time in BST
                current_time_bst = datetime.now(bst)

                # Fetch the current hour in BST
                current_hour_bst = current_time_bst.hour

                # Fetch the last row's 'Scheduled Time' from the updated cumulative_df
                last_row_scheduled_time = cumulative_df_new['Scheduled Time'].iloc[-1]

                # Convert the last_row_scheduled_time to BST if it's not already
                # Assuming last_row_scheduled_time is naive (no timezone info)
                last_row_scheduled_time_bst = last_row_scheduled_time.replace(tzinfo=bst)

                # Print the last row's 'Scheduled Time'
                #print("\nLast row's 'Scheduled Time' in cumulative_df (BST):")
                #print(last_row_scheduled_time_bst)

                # Fetch the scheduled times from timetable_df that meet the criteria
                filtered_times = timetable_df[
                    (timetable_df['Scheduled Time Only'] > last_row_scheduled_time_bst) &
                    (timetable_df['Scheduled Time Only'].apply(lambda x: x.hour).isin([current_hour_bst, current_hour_bst + 1, current_hour_bst + 2]))
                ]

                # Print the filtered scheduled times
                #print("\nScheduled times greater than last_row_scheduled_time in BST and within the next 3 hours:")
                #print(filtered_times['Scheduled Time Only'])

                # Step 1: Find the shape of `filtered_times['Scheduled Time Only']`
                filtered_times_shape = filtered_times['Scheduled Time Only'].shape[0]

                # Step 2: Create new rows with the filtered scheduled times
                new_rows = pd.DataFrame({
                    'Scheduled Time': filtered_times['Scheduled Time Only'].values,
                    'Line': [cumulative_df_new['Line'].iloc[-1]] * filtered_times_shape,
                    'Stop Point': [cumulative_df_new['Stop Point'].iloc[-1]] * filtered_times_shape,
                    'Direction': [cumulative_df_new['Direction'].iloc[-1]] * filtered_times_shape,
                    'Vehicle ID': [f"{str(cumulative_df_new['Stop Point'].iloc[-1])[-4:]}_{i+1}" for i in range(filtered_times_shape)],
                    'Expected Arrival (BST)': [0] * filtered_times_shape,
                    'Expected Arrival (HM)': [0] * filtered_times_shape,
                    'Gap': [0] * filtered_times_shape,
                    '2_Gap': [0] * filtered_times_shape,
                    'Gap_Sq': [0] * filtered_times_shape
                })

                # Step 3: Append the new rows to the `cumulative_df`
                cumulative_df_new = pd.concat([cumulative_df_new, new_rows], ignore_index=True)

                # Step 4: Print the updated cumulative_df
                #print("\nUpdated cumulative_df after appending new rows:")
                #print(cumulative_df_new)

                #print("\nData types of each column in cumulative_df_new:")
                #print(cumulative_df_new.dtypes)



                # Function to convert time string to seconds since start of day
                def time_to_seconds(time_input):
                    if isinstance(time_input, str):
                        # Parse string to datetime.time
                        t = datetime.strptime(time_input, '%H:%M:%S').time()
                    elif isinstance(time_input, dt_time):
                        # If it's already a datetime.time object
                        t = time_input
                    else:
                        raise TypeError(f"Unexpected type: {type(time_input)}")
                    return t.hour * 3600 + t.minute * 60 + t.second

                # Function to convert seconds to time string with current date
                def seconds_to_time(seconds, date_str):
                    return (datetime.strptime(date_str, '%Y-%m-%d') + timedelta(seconds=seconds)).strftime('%Y-%m-%d %H:%M:%S')

                # Function to format the time string to hours and minutes with `00` for seconds
                def format_time_str(time_str):
                    try:
                        return f"{time_str.split(':')[0]}:{time_str.split(':')[1]}:00"
                    except IndexError:
                        return '00:00:00'

                # Convert 'Scheduled Time' and 'Expected Arrival (BST)' in the training data to seconds
                # Function to convert time string or Timestamp to seconds
                def convert_to_seconds(time_input):
                    if isinstance(time_input, str):
                        try:
                            # Extract the time part from the string
                            time_part = time_input.split()[1]
                            return time_to_seconds(time_part)
                        except IndexError:
                            return np.nan
                    elif isinstance(time_input, pd.Timestamp):
                        # Convert Timestamp to time string
                        time_str = time_input.time().strftime('%H:%M:%S')
                        return time_to_seconds(time_str)
                    elif isinstance(time_input, int):
                        # If it's already an integer, assume it's in seconds
                        return time_input
                    else:
                        raise TypeError(f"Unexpected type: {type(time_input)}")

                # Load the training data from Google Drive
                file_path_train = '/content/drive/My Drive/Files/Training_set.csv'
                df_train = pd.read_csv(file_path_train)

                #printing new_data
                #print("\nprinting df train1:")
                #print(df_train)

                # Convert 'Scheduled Time' and 'Expected Arrival (BST)' in the training data to seconds
                df_train['Scheduled Time (seconds)'] = df_train['Scheduled Time'].apply(time_to_seconds)

                # Convert 'Expected Arrival (BST)' to seconds, handling cases where the value doesn't split correctly
                df_train['Expected Arrival (BST) (seconds)'] = df_train['Expected Arrival (BST)'].apply(time_to_seconds)


                #printing new_data
                #print("\nprinting df train2:")
                #print(df_train)

                # Filter out rows with NaN in 'Expected Arrival (BST) (seconds)' column
                df_train = df_train.dropna(subset=['Expected Arrival (BST) (seconds)'])

                #printing new_data
                #print("\nprinting df train3:")
                #print(df_train)


                # Assuming cumulative_df_new is already defined and available
                # Filter cumulative_df_new to get rows where Expected Arrival (BST) is not zero
                # For numeric data types (int or float)
                non_zero_df = cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'] != 0]

                #print("\nprinting non_zero_df:")
                #print(non_zero_df)

                # Extract scheduled time and expected arrival from cumulative_df_new into new_data DataFrame
                new_data = pd.DataFrame({
                    'Scheduled Time': non_zero_df['Scheduled Time'],
                    'Expected Arrival (BST)': non_zero_df['Expected Arrival (BST)']
                })

                #print("\nprinting new data 1:")
                #print(new_data)


                # Convert new_data columns to seconds
                new_data['Scheduled Time (seconds)'] = new_data['Scheduled Time'].apply(time_to_seconds)
                new_data['Expected Arrival (BST) (seconds)'] = new_data['Expected Arrival (BST)'].apply(convert_to_seconds)

                #print("\nprinting new data 2:")
                #print(new_data)

                # Filter out rows with NaN in the new_data DataFrame
                new_data = new_data.dropna(subset=['Expected Arrival (BST) (seconds)'])
                #print("\nprinting new data 3:")
                #print(new_data)



                # Append new_data to your existing DataFrame for training
                df_train = pd.concat([df_train, new_data], ignore_index=True)
                #print("\nprinting df train4:")
                #print(df_train)

                # Define features (X) and target (y) for training
                X = df_train[['Scheduled Time (seconds)']]
                y = df_train['Expected Arrival (BST) (seconds)']

                #print("\nprinting X :")
                #print(X)
                #print("\nprinting y:")
                #print(y)
                # Initialize and train the Linear Regression model
                model = LinearRegression()
                model.fit(X, y)

                # Extract scheduled times where Expected Arrival (BST) is zero
                zero_arrival_df = cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'] == 0]
                #print("\nprinting zero_arrival_df:")
                #print(zero_arrival_df)

                #new_scheduled_times = zero_arrival_df['Scheduled Time'].tolist()
                #print("\nprinting new_scheduled_times1:")
                #print(new_scheduled_times)
                new_scheduled_times = [str(time).zfill(8) for time in zero_arrival_df['Scheduled Time']]
                #print("\nprinting new_scheduled_times2:")
                #print(new_scheduled_times)
                # Convert new scheduled times to seconds
                new_scheduled_times_seconds = [time_to_seconds(t) for t in new_scheduled_times]
                #print("\nprinting new_scheduled_times3:")
                #print(new_scheduled_times)
                # Predict expected arrival times
                predicted_seconds = model.predict(pd.DataFrame({'Scheduled Time (seconds)': new_scheduled_times_seconds}))
                #print("\nprinting predicted_seconds:")
                #print(predicted_seconds)
                # Adjust predictions
                adjusted_predictions_seconds = []
                previous_expected_seconds = new_scheduled_times_seconds[0]
                #print("\nprinting previous_expected_seconds:")
                #print(previous_expected_seconds)

                #for i, (scheduled_sec, predicted_sec) in enumerate(zip(new_scheduled_times_seconds, predicted_seconds)):
                    #if predicted_sec <= scheduled_sec:
                        #predicted_sec = scheduled_sec + 60
                    #if i > 0 and predicted_sec <= adjusted_predictions_seconds[-1]:
                        #predicted_sec = adjusted_predictions_seconds[-1] + 60
                    #adjusted_predictions_seconds.append(predicted_sec)


                for i, (scheduled_sec, predicted_sec) in enumerate(zip(new_scheduled_times_seconds, predicted_seconds)):
                    while predicted_sec <= scheduled_sec or (i > 0 and predicted_sec <= adjusted_predictions_seconds[-1]):
                        predicted_sec += 60  # Add 1 minute
                    adjusted_predictions_seconds.append(predicted_sec)

                # Get current date
                current_date = datetime.now().strftime('%Y-%m-%d')

                # Convert adjusted predictions to time strings with the current date
                adjusted_times = [seconds_to_time(sec, current_date) for sec in adjusted_predictions_seconds]
                #print("\nprinting adjusted_times:")
                #print(adjusted_times)
                # Update cumulative_df_new with the adjusted times
                cumulative_df_new.loc[zero_arrival_df.index, 'Expected Arrival (BST)'] = adjusted_times
                #print("\nprinting cumulative_df with adjusted_times2:")
                #print(cumulative_df_new)
                # Update 'Expected Arrival (HM)' for rows where value is '0'
                # Extract the date from the first non-zero 'Expected Arrival (HM)' element
                #first_hm_date = cumulative_df_new.loc[cumulative_df_new['Expected Arrival (HM)'] != '0', 'Expected Arrival (HM)'].iloc[0].split()[0]
                first_hm_date = '1900-01-01'
                #print("\nprinting first_hm_date:")
                #print(first_hm_date)
                #print("\nData type of first_hm_date:", type(first_hm_date))
                def update_hm(row):
                    if row['Expected Arrival (HM)'] == 0:
                        bst_time = row['Expected Arrival (BST)'].split()[1]
                        return f"{first_hm_date} {format_time_str(bst_time)}"
                    return row['Expected Arrival (HM)']
                #print("\nprinting cumulative_df with before Expected arrival HM modified:")
                #print(cumulative_df_new)
                cumulative_df_new['Expected Arrival (HM)'] = cumulative_df_new.apply(update_hm, axis=1)
                #print("\nprinting cumulative_df with Expected arrival HM modified:")
                #print(cumulative_df_new)
                # Select the last 10 columns
                #cumulative_df_new_10 = cumulative_df_new.iloc[:, -10:]


                #print("\nprinting cumulative_df with last 10 columns:")
                #print(cumulative_df_new_10)



                # Calculate 'Gap', '2_Gap', and 'Gap_Sq'
                # Convert 'Expected Arrival (BST)' to datetime
                cumulative_df_new['Expected Arrival (BST)'] = pd.to_datetime(cumulative_df_new['Expected Arrival (BST)'], format='%Y-%m-%d %H:%M:%S')

                # Calculate 'Gap' as the difference between successive 'Expected Arrival (BST)' times in minutes
                cumulative_df_new['Gap'] = cumulative_df_new['Expected Arrival (BST)'].diff().dt.total_seconds() / 60
                cumulative_df_new['Gap'] = cumulative_df_new['Gap'].round(2)

                # Set the first row 'Gap' to 0
                cumulative_df_new.loc[0, 'Gap'] = 0.0

                # Calculate '2_Gap' as twice the 'Gap'
                cumulative_df_new['2_Gap'] = (cumulative_df_new['Gap'] * 2).round(2)

                # Calculate 'Gap_Sq' as the square of the 'Gap'
                cumulative_df_new['Gap_Sq'] = (cumulative_df_new['Gap'] * cumulative_df_new['Gap']).round(2)

                # Print the updated cumulative_df_new with calculated values
                print("\nprinting cumulative_df with predicted Expected Arrival (BST):")
                #cumulative_df_new = cumulative_df_new.sort_values(by='Expected Arrival (BST)')
                print(cumulative_df_new)

                current_hour = datetime.now(pytz.timezone('Europe/London')).hour
                next_hour = (current_hour + 1)
                second_next_hour = (current_hour + 2)
                #print(f"current_hour: {current_hour}")
                #print(f"next_hour: {next_hour}")
                #print(f"second_next_hour: {second_next_hour}")
                # Update number of buses observed in the current hour
                num_buses_observed_next1 = len(cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'].dt.hour == next_hour]['Vehicle ID'].unique())
                num_buses_observed_next2 = len(cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'].dt.hour == second_next_hour]['Vehicle ID'].unique())
                #print(f"buses forecasted for next_hour: {num_buses_observed_next1}")
                #print(f"buses forecasted for second_next_hour: {num_buses_observed_next2}")

                total_Gap_Sq_next1 = cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'].dt.hour == next_hour]['Gap_Sq'].sum().round(2)
                total_Gap_Sq_next2 = cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'].dt.hour == second_next_hour]['Gap_Sq'].sum()
                #print(f"total_Gap_Sq_next1: {total_Gap_Sq_next1}")
                #print(f"total_Gap_Sq_next2: {total_Gap_Sq_next2}")

                total_2_Gap_next1 = cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'].dt.hour == next_hour]['2_Gap'].sum().round(2)
                total_2_Gap_next2 = cumulative_df_new[cumulative_df_new['Expected Arrival (BST)'].dt.hour == second_next_hour]['2_Gap'].sum()
                #print(f"total_2_Gap_next1: {total_2_Gap_next1}")
                #print(f"total_2_Gap_next2: {total_2_Gap_next2}")

                AWT_next1 = round(total_Gap_Sq_next1 / total_2_Gap_next1, 2) if total_2_Gap_next1 > 0 else 0
                AWT_next2 = round(total_Gap_Sq_next2 / total_2_Gap_next2, 2) if total_2_Gap_next2 > 0 else 0
                #print(f"AWT_next1: {AWT_next1}")
                #print(f"AWT_next2: {AWT_next2}")

                buses_observed_new[stop_point_id] = (num_buses_observed_next1, total_Gap_Sq_next1, total_2_Gap_next1, AWT_next1)
                buses_observed_new2[stop_point_id] = (num_buses_observed_next2, total_Gap_Sq_next2, total_2_Gap_next2, AWT_next2)
                #print(f"buses_observed_new: {buses_observed_new}")
                #print(f"buses_observed_new2: {buses_observed_new2}")

        print(f"\nRoute {lineID} SWT DataFrame:")
        print(route_swt_df)
        double_headway_a, double_headway_b = fetch_current_hour_swt(route_swt_df)

        if double_headway_a is not None and double_headway_b is not None:
          results_df = check_gaps_against_double_headway(cumulative_dataframes, double_headway_a, double_headway_b, combined_df, lineID)

          #print("\nVehicle IDs and Stop Points where Gap exceeds Double Headway:")
          #print(results_df.to_string(index=False))

        # Create DataFrame to show number of buses observed for each stop point
        buses_observed_df = pd.DataFrame(list(buses_observed.items()), columns=['Stop Point', 'Metrics'])
        buses_observed_df_new = pd.DataFrame(list(buses_observed_new.items()), columns=['Stop Point', 'Metrics'])
        buses_observed_df_new2 = pd.DataFrame(list(buses_observed_new2.items()), columns=['Stop Point', 'Metrics'])

        # Split 'Metrics' into separate columns
        buses_observed_df[['Num of Buses Observed', 'Total Gap Sq', 'Total 2 Gap', 'AWT']] = pd.DataFrame(
            buses_observed_df['Metrics'].tolist(), index=buses_observed_df.index
        )

        buses_observed_df_new[['Num of Buses Observed', 'Total Gap Sq', 'Total 2 Gap', 'AWT']] = pd.DataFrame(
            buses_observed_df_new['Metrics'].tolist(), index=buses_observed_df_new.index
        )

        buses_observed_df_new2[['Num of Buses Observed', 'Total Gap Sq', 'Total 2 Gap', 'AWT']] = pd.DataFrame(
            buses_observed_df_new2['Metrics'].tolist(), index=buses_observed_df_new2.index
        )

        # Calculate WAWT as the product of AWT and Num of Buses Observed
        buses_observed_df['WAWT'] = buses_observed_df['AWT'] * buses_observed_df['Num of Buses Observed']
        buses_observed_df_new['WAWT'] = buses_observed_df_new['AWT'] * buses_observed_df_new['Num of Buses Observed']
        buses_observed_df_new2['WAWT'] = buses_observed_df_new2['AWT'] * buses_observed_df_new2['Num of Buses Observed']

        # Drop the 'Metrics' column
        buses_observed_df.drop(columns=['Metrics'], inplace=True)
        buses_observed_df_new.drop(columns=['Metrics'], inplace=True)
        buses_observed_df_new2.drop(columns=['Metrics'], inplace=True)

        current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        next_hour = (current_hour + 1)
        second_next_hour = (current_hour + 2)

        buses_observed_df['Line'] = lineID
        buses_observed_df['Hour'] = current_hour

        buses_observed_df_new['Line'] = lineID
        buses_observed_df_new['Hour'] = next_hour


        buses_observed_df_new2['Line'] = lineID
        buses_observed_df_new2['Hour'] = second_next_hour

        # Reorder columns to make 'Hour' the first column
        columns_order = ['Line'] + ['Hour'] + [col for col in buses_observed_df.columns if col not in ['Hour', 'Line']]
        buses_observed_df = buses_observed_df[columns_order]


        columns_order_new = ['Line'] + ['Hour'] + [col for col in buses_observed_df_new.columns if col not in ['Hour', 'Line']]
        buses_observed_df_new = buses_observed_df_new[columns_order_new]

        columns_order_new = ['Line'] + ['Hour'] + [col for col in buses_observed_df_new2.columns if col not in ['Hour', 'Line']]
        buses_observed_df_new2 = buses_observed_df_new2[columns_order_new]

        print(f"\nNumber of Buses Observed DataFrame for Route:{lineID}")
        print(buses_observed_df)

        print(f"\nNumber of Buses Forecasted DataFrame for Route in next hour:{lineID}")
        print(buses_observed_df_new)

        print(f"\nNumber of Buses Forecasted DataFrame for Route in second next hour:{lineID}")
        print(buses_observed_df_new2)

        # Calculate Route AWT A and Route AWT B
        pattern_A = f"^{lineID}_A\\d+$"
        pattern_B = f"^{lineID}_B\\d+$"

        buses_observed_df_A = buses_observed_df[buses_observed_df['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_A)]['ID'])]
        buses_observed_df_B = buses_observed_df[buses_observed_df['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_B)]['ID'])]

        buses_observed_df_A_new = buses_observed_df_new[buses_observed_df_new['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_A)]['ID'])]
        buses_observed_df_B_new = buses_observed_df_new[buses_observed_df_new['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_B)]['ID'])]

        buses_observed_df_A_new2 = buses_observed_df_new2[buses_observed_df_new2['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_A)]['ID'])]
        buses_observed_df_B_new2 = buses_observed_df_new2[buses_observed_df_new2['Stop Point'].isin(combined_df[combined_df['Route_Dir_QSI_No'].str.match(pattern_B)]['ID'])]

        sum_WAWT_A = buses_observed_df_A['WAWT'].sum()
        sum_buses_observed_A = buses_observed_df_A['Num of Buses Observed'].sum()
        route_AWT_A = round(sum_WAWT_A / sum_buses_observed_A, 2) if sum_buses_observed_A > 0 else 0

        sum_WAWT_A_new = buses_observed_df_A_new['WAWT'].sum()
        sum_buses_observed_A_new = buses_observed_df_A_new['Num of Buses Observed'].sum()
        route_AWT_A_new = round(sum_WAWT_A_new / sum_buses_observed_A_new, 2) if sum_buses_observed_A_new > 0 else 0

        sum_WAWT_A_new2 = buses_observed_df_A_new2['WAWT'].sum()
        sum_buses_observed_A_new2 = buses_observed_df_A_new2['Num of Buses Observed'].sum()
        route_AWT_A_new2 = round(sum_WAWT_A_new2 / sum_buses_observed_A_new2, 2) if sum_buses_observed_A_new2 > 0 else 0

        sum_WAWT_B = buses_observed_df_B['WAWT'].sum()
        sum_buses_observed_B = buses_observed_df_B['Num of Buses Observed'].sum()
        route_AWT_B = round(sum_WAWT_B / sum_buses_observed_B, 2) if sum_buses_observed_B > 0 else 0

        sum_WAWT_B_new = buses_observed_df_B_new['WAWT'].sum()
        sum_buses_observed_B_new = buses_observed_df_B_new['Num of Buses Observed'].sum()
        route_AWT_B_new = round(sum_WAWT_B_new / sum_buses_observed_B_new, 2) if sum_buses_observed_B_new > 0 else 0

        sum_WAWT_B_new2 = buses_observed_df_B_new2['WAWT'].sum()
        sum_buses_observed_B_new2 = buses_observed_df_B_new2['Num of Buses Observed'].sum()
        route_AWT_B_new2 = round(sum_WAWT_B_new2 / sum_buses_observed_B_new2, 2) if sum_buses_observed_B_new2 > 0 else 0

        if 'Route' not in route_awt_df.columns:
          route_awt_df['Route'] = lineID

        if 'Route' not in route_awt_df_new.columns:
          route_awt_df_new['Route'] = lineID

        if 'Route' not in route_awt_df_new2.columns:
          route_awt_df_new2['Route'] = lineID

        # Check if the current hour's data is already present
        if current_hour in route_awt_df['Hour'].values:
            route_awt_df.loc[route_awt_df['Hour'] == current_hour, ['Route AWT A', 'Route AWT B']] = [route_AWT_A, route_AWT_B]
        else:
            new_row = pd.DataFrame({
                'Route': [lineID],
                'Hour': [current_hour],
                'Route AWT A': [route_AWT_A],
                'Route AWT B': [route_AWT_B]
            })
            route_awt_df = pd.concat([route_awt_df, new_row], ignore_index=True)



        # Check if the next hour's data is already present
        if next_hour in route_awt_df_new['Hour'].values:
            route_awt_df_new.loc[route_awt_df_new['Hour'] == next_hour, ['Route AWT A', 'Route AWT B']] = [route_AWT_A_new, route_AWT_B_new]
        else:
            new_row_new = pd.DataFrame({
                'Route': [lineID],
                'Hour': [next_hour],
                'Route AWT A': [route_AWT_A_new],
                'Route AWT B': [route_AWT_B_new]
            })
            route_awt_df_new = pd.concat([route_awt_df_new, new_row_new], ignore_index=True)

        # Check if the second next hour's data is already present
        if second_next_hour in route_awt_df_new2['Hour'].values:
            route_awt_df_new2.loc[route_awt_df_new2['Hour'] == second_next_hour, ['Route AWT A', 'Route AWT B']] = [route_AWT_A_new2, route_AWT_B_new2]
        else:
            new_row_new2 = pd.DataFrame({
                'Route': [lineID],
                'Hour': [second_next_hour],
                'Route AWT A': [route_AWT_A_new2],
                'Route AWT B': [route_AWT_B_new2]
            })
            route_awt_df_new2 = pd.concat([route_awt_df_new2, new_row_new2], ignore_index=True)

        columns_order = ['Route'] + [col for col in route_awt_df.columns if col != 'Route']
        route_awt_df = route_awt_df[columns_order]

        columns_order_new = ['Route'] + [col for col in route_awt_df_new.columns if col != 'Route']
        route_awt_df_new = route_awt_df_new[columns_order_new]

        columns_order_new2 = ['Route'] + [col for col in route_awt_df_new2.columns if col != 'Route']
        route_awt_df_new2 = route_awt_df_new2[columns_order_new2]


        route_ewt_df = pd.DataFrame(columns=['Route','Hour', 'Route EWT A', 'Route EWT B'])
        #current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        route_ewt_df = pd.DataFrame([[lineID,current_hour, None, None]], columns=['Route','Hour', 'Route EWT A','Route EWT B'])

        route_ewt_df_new = pd.DataFrame(columns=['Route','Hour', 'Route EWT A', 'Route EWT B'])
        #current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        route_ewt_df_new = pd.DataFrame([[lineID,next_hour, None, None]], columns=['Route','Hour', 'Route EWT A','Route EWT B'])


        route_ewt_df_new2 = pd.DataFrame(columns=['Route','Hour', 'Route EWT A', 'Route EWT B'])
        #current_hour = datetime.now(pytz.timezone('Europe/London')).hour
        route_ewt_df_new2 = pd.DataFrame([[lineID,second_next_hour, None, None]], columns=['Route','Hour', 'Route EWT A','Route EWT B'])


        # Merge route_awt_df and route_swt_df on 'Hour'
        merged_df = pd.merge(route_awt_df, route_swt_df, on='Hour')

        merged_df_new = pd.merge(route_awt_df_new, route_swt_df, on='Hour')

        merged_df_new2 = pd.merge(route_awt_df_new2, route_swt_df, on='Hour')

        # Calculate the 'Route EWT A' and 'Route EWT B' columns
        merged_df['Route EWT A'] = merged_df['Route AWT A'] - merged_df['Route SWT A']
        merged_df['Route EWT B'] = merged_df['Route AWT B'] - merged_df['Route SWT B']

        # Calculate the 'Route EWT A' and 'Route EWT B' columns of next hour
        merged_df_new['Route EWT A'] = merged_df_new['Route AWT A'] - merged_df_new['Route SWT A']
        merged_df_new['Route EWT B'] = merged_df_new['Route AWT B'] - merged_df_new['Route SWT B']


        # Calculate the 'Route EWT A' and 'Route EWT B' columns of second next hour
        merged_df_new2['Route EWT A'] = merged_df_new2['Route AWT A'] - merged_df_new2['Route SWT A']
        merged_df_new2['Route EWT B'] = merged_df_new2['Route AWT B'] - merged_df_new2['Route SWT B']


        lineID_sheet2 = str(lineID)
        df_sheet2['Route'] = df_sheet2['Route'].astype(str)
        MPS_data = df_sheet2[(df_sheet2['Route'] == lineID_sheet2) | (df_sheet2['Route'] == lineID)]
        MPS = MPS_data['MPS'].iloc[0]
        print(f"\n\033[1m\033[4mMPS for Route {lineID} is {MPS}\033[0m\n")

        # Calculate the 'Route EWT (var) A' and 'Route EWT (var) B' columns
        merged_df['Route EWT VAR A'] = merged_df['Route EWT A'] - MPS
        merged_df['Route EWT VAR B'] = merged_df['Route EWT B'] - MPS

        # Calculate the 'Route EWT (var) A' and 'Route EWT (var) B' columns of next hour
        merged_df_new['Route EWT VAR A'] = merged_df_new['Route EWT A'] - MPS
        merged_df_new['Route EWT VAR B'] = merged_df_new['Route EWT B'] - MPS

        # Calculate the 'Route EWT (var) A' and 'Route EWT (var) B' columns of second next hour
        merged_df_new2['Route EWT VAR A'] = merged_df_new2['Route EWT A'] - MPS
        merged_df_new2['Route EWT VAR B'] = merged_df_new2['Route EWT B'] - MPS

        # Reorder the columns so that 'Route EWT A' and 'Route EWT B' are after 'Route SWT B'
        new_columns_order = ['Route', 'Hour', 'Route SWT A', 'Route AWT A', 'Route EWT A', 'Route EWT VAR A', 'Route SWT B', 'Route AWT B', 'Route EWT B' , 'Route EWT VAR B']
        merged_df = merged_df[new_columns_order]
        merged_df_new = merged_df_new[new_columns_order]
        merged_df_new2 = merged_df_new2[new_columns_order]

        # Split the DataFrame for Route A and Route B
        route_A_df = merged_df[['Route','Hour', 'Route SWT A', 'Route AWT A', 'Route EWT A', 'Route EWT VAR A']]
        route_B_df = merged_df[['Route','Hour', 'Route SWT B', 'Route AWT B', 'Route EWT B', 'Route EWT VAR B']]

        # Split the DataFrame for Route A and Route B
        route_A_df_new = merged_df_new[['Route','Hour', 'Route SWT A', 'Route AWT A', 'Route EWT A', 'Route EWT VAR A']]
        route_B_df_new = merged_df_new[['Route','Hour', 'Route SWT B', 'Route AWT B', 'Route EWT B', 'Route EWT VAR B']]

        # Split the DataFrame for Route A and Route B
        route_A_df_new2 = merged_df_new2[['Route','Hour', 'Route SWT A', 'Route AWT A', 'Route EWT A', 'Route EWT VAR A']]
        route_B_df_new2 = merged_df_new2[['Route','Hour', 'Route SWT B', 'Route AWT B', 'Route EWT B', 'Route EWT VAR B']]

        route_A_df = route_A_df.rename(columns={'Route': 'Line'})
        route_B_df = route_B_df.rename(columns={'Route': 'Line'})


        route_A_df_new = route_A_df_new.rename(columns={'Route': 'Line'})
        route_B_df_new = route_B_df_new.rename(columns={'Route': 'Line'})

        route_A_df_new2 = route_A_df_new2.rename(columns={'Route': 'Line'})
        route_B_df_new2 = route_B_df_new2.rename(columns={'Route': 'Line'})

        print(f"\nSWT, AWT, and EWT of Route {lineID} in direction A:")
        print(route_A_df)

        print(f"\nSWT, AWT, and EWT of Route {lineID} in direction B:")
        print(route_B_df)

        print(f"\nSWT, AWT, and EWT of Route {lineID} in direction A for next hour:")
        print(route_A_df_new)

        print(f"\nSWT, AWT, and EWT of Route {lineID} in direction B for next hour:")
        print(route_B_df_new)

        print(f"\nSWT, AWT, and EWT of Route {lineID} in direction A for second next hour:")
        print(route_A_df_new2)

        print(f"\nSWT, AWT, and EWT of Route {lineID} in direction B for second next hour:")
        print(route_B_df_new2)

        # Save DataFrames to CSV
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Save DataFrames to Google Drive
        route_swt_df.to_csv(os.path.join(output_dir, 'route_swt_df.csv'), index=False)
        results_df.to_csv(os.path.join(output_dir, 'results_df.csv'), index=False)
        route_A_df.to_csv(os.path.join(output_dir, 'route_A_df.csv'), index=False)
        route_B_df.to_csv(os.path.join(output_dir, 'route_B_df.csv'), index=False)


        print(f"CSV files saved to 'output' directory.\n")

if __name__ == "__main__":
    main()



Today is Saturday. The selected Schedule name is Saturday.

Arrival Predictions for stop point 490011107G (Poplar / All Saints Church):
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
  D7    LX61DBO 490011107G  outbound    2024-08-17 13:28:38   1900-01-01 13:28:00  0.0    0.0     0.0

Cumulative DataFrame 1:
Line Vehicle ID Stop Point Direction Expected Arrival (BST) Expected Arrival (HM)  Gap  2_Gap  Gap_Sq
  D7    LX61DBO 490011107G  outbound    2024-08-17 13:28:38   1900-01-01 13:28:00  0.0    0.0     0.0

Number of buses observed in the current hour: 1
Refreshing data in 30 seconds...



KeyboardInterrupt: 